In [1]:
import pickle
import cv2
import os
import json
import csv
import numpy as np

In [2]:
with open('../../temp/oi_600_hierarchy.pkl', 'rb') as fin:
    hierarchy = pickle.load(fin)

In [3]:
with open('../../../wsod/metadata/ont_300/300-classes.csv', 'r') as fin:
    labels_filter = [item.strip() for item in fin]

In [4]:
with open('../../temp/all_labels_names.pkl', 'rb') as fin:
    mid2name = pickle.load(fin)

In [5]:
with open('../../results/det_results_merged_20b.pkl', 'rb') as fin:
    det_results = pickle.load(fin)

In [6]:
label_map = {}
for key in mid2name:
    label = key
    print(f'looking for {mid2name.get(label)} ({label})... ', end='')
    while label not in labels_filter:
        label = hierarchy.get(label)
        if label == None: 
            break
        print(f'changed to {mid2name.get(label)} ({label}); ', end='')
    if label == None:
        print(f'NOT found.')
        continue
    print(f'found.')
    label_map[key] = label

looking for toaster (/m/01k6s3)... changed to Home appliance (/m/019dx1); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Backpack (/m/01940j)... changed to Luggage and bags (/m/0hf58v5); found.
looking for Sandwich (/m/0l515)... changed to Food (/m/02wbm); found.
looking for Elephant (/m/0bwd_0j)... changed to Mammal (/m/04rky); changed to Animal (/m/0jbk); found.
looking for Houseplant (/m/03fp41)... changed to Plant (/m/05s2s); found.
looking for Teddy bear (/m/0kmg4)... changed to Toy (/m/0138tl); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Skateboard (/m/06_fw)... changed to Sports equipment (/m/05y5lj); changed to None (/virtual); changed to None (/m/0bl9f); NOT found.
looking for Vase (/m/02s195)... changed to None (/m/0h8nzzj); changed to None (/j/cjpcdr); changed to None (/m/0bl9f); NOT found.
looking for Chair (/m/01mzpv)... changed to Furniture (/m/0c_jw); found.
looking for Microwave oven (/m/0fx9l)... chan

In [7]:
det_results_filtered = {}
for key, val in det_results.items():
    det_results_filtered[key] = []
    for det in val:
        label = label_map.get(det['label'])
        if label == None:
            continue
        det_results_filtered[key].append({
            'label': label,
            'score': det['score'],
            'bbox': det['bbox'],
            'bbox_normalized': det['bbox_normalized'],
            'model': det['model'],            
        })


In [8]:
with open('../../results/det_results_postproc_20b.pkl', 'wb') as fout:
    pickle.dump(det_results_filtered, fout)

## Sorting the results

In [9]:
det_results_sorted = {}
for imgid, det in det_results_filtered.items():
    detection_classes = [item['label'] for item in det]
    detection_scores = [item['score'] for item in det]
    detection_boxes = [np.asarray(item['bbox']) for item in det]
    detection_boxes_n = [np.asarray(item['bbox_normalized']) for item in det]
    detection_models = [item['model'] for item in det]

    sort_idx = np.argsort(-np.asarray(detection_scores))
    
    detection_classes = [detection_classes[i] for i in sort_idx]
    detection_scores = [detection_scores[i] for i in sort_idx]
    detection_boxes = [detection_boxes[i] for i in sort_idx]
    detection_boxes_n = [detection_boxes_n[i] for i in sort_idx]
    detection_models = [detection_models[i] for i in sort_idx]
    
    det_results_sorted[imgid] = {
        'detection_classes': detection_classes,
        'detection_scores': detection_scores,
        'detection_boxes': detection_boxes,
        'detection_boxes_normalized': detection_boxes_n,
        'detection_models': detection_models,            
        'num_detections': len(detection_classes)
    }
    

In [10]:
with open('../../results/det_results_postproc_20_sorted.pkl', 'wb') as fout:
    pickle.dump(det_results_sorted, fout)